# 04 - Summarize & Translate Agent
## SEEDS Nexus AI Agents Academy

Build your first AI agent with specialized tools! In this final notebook, you'll create a functional AI agent that can summarize environmental content and translate it into multiple languages.

### Learning Objectives
- Build custom text processing tools
- Create a summarization tool for environmental content
- Develop a translation tool for multiple languages
- Combine tools into a decision-making agent
- Test the agent with real environmental data

### Environmental Focus
Your agent will help make environmental information more accessible by summarizing complex topics and breaking down language barriers.

In [ ]:
# Setup Cell - Install Required Packages
# Run this cell first in Google Colab

!pip install langchain==0.1.0
!pip install langchain-openai==0.0.5
!pip install langchain-community==0.0.10
!pip install openai==1.12.0
!pip install tiktoken==0.5.2
!pip install python-dotenv==1.0.0

## Setup: API Key Configuration

Before we start building our agent, we need to set up our OpenAI API key. The cell below will:
- Check if you're running in Google Colab or locally
- Set up your API key securely based on the environment
- Verify the API key is properly configured

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running locally")

# Set up OpenAI API key based on environment
if IN_COLAB:
    # For Google Colab: use the secure input method
    from google.colab import userdata
    from getpass import getpass

    try:
        # Try to get API key from Colab secrets first
        openai_api_key = userdata.get('OPENAI_API_KEY')
        if openai_api_key:
            os.environ["OPENAI_API_KEY"] = openai_api_key
            print("✅ API key loaded from Google Colab secrets!")
        else:
            # If not in secrets, prompt user to enter it
            print("OpenAI API key not found in Colab secrets.")
            os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
            print("✅ API key set from input")
    except Exception as e:
        print(f"Note: {e}")
        print("Enter your OpenAI API key below:")
        os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")
else:
    # For local environment: try to load from .env file
    try:
        from dotenv import load_dotenv
        load_dotenv()
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            print("✅ API key loaded from .env file")
        else:
            print("⚠️ No API key found in .env file. Please add your OpenAI API key.")
            os.environ["OPENAI_API_KEY"] = "your-api-key-here"
    except ImportError:
        print("⚠️ python-dotenv not installed. Please install it or add your API key manually.")
        os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Check if API key is properly set
if os.environ.get("OPENAI_API_KEY") in [None, "", "your-api-key-here"]:
    print("⚠️ WARNING: Please set your OpenAI API key before running the examples!")
    print("Get your API key from: https://platform.openai.com/api-keys")
    if IN_COLAB:
        print("For Colab: Use Secrets to securely store your API key")
        print("  1. Click on the 🔑 icon in the left sidebar")
        print("  2. Add a new secret with name 'OPENAI_API_KEY'")
        print("  3. Run this cell again")
    else:
        print("For local use: Create a .env file with OPENAI_API_KEY=your-key-here")
else:
    print("✅ API key is set! Ready to proceed.\n")

## Step 1: Import Required Libraries

First, let's import the libraries we'll need to create our agent:

In [ ]:
# Import required libraries
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

print("✅ Libraries imported successfully!")

## Step 2: Create the Language Model

We'll use OpenAI's GPT model as the brain of our agent. This will power both our summarization and translation tools.

In [ ]:
# Create our language model
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
print("✅ Language model initialized")
print("Note: We're using temperature=0 for more consistent, focused responses.")

## Step 3: Create the Summarization Tool

Let's build our first tool - a summarization tool that can condense environmental text into key bullet points. This will help make complex environmental information more digestible.

In [ ]:
# Create a summarization tool
summarize_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    Summarize the following text in 3-4 concise bullet points, focusing on key environmental and sustainability aspects:

    {text}

    Summary:
    """
)

# Create a chain that will process our text
summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt)

# Define a function that the agent can call
def summarize_text(text):
    """Summarize environmental text into bullet points"""
    return summarize_chain.run(text=text)

print("✅ Summarization tool created!")

## Step 4: Test the Summarization Tool

Let's test our summarization tool with some environmental text to see how it works.

In [ ]:
# Test sample text about renewable energy
environmental_text = """
Renewable energy is transforming the global energy landscape. Solar power capacity has increased by 85% since 2010,
while wind energy has seen a 70% growth in the same period. This transition is critical because fossil fuels account
for approximately 75% of global greenhouse gas emissions. By switching to renewable sources, we can reduce carbon
footprints significantly. Additionally, renewable energy projects create three times more jobs than fossil fuel
investments. Despite this progress, challenges remain in energy storage, grid integration, and policy support.
"""

# Test our summarization tool
summary = summarize_text(environmental_text)
print("Original text length:", len(environmental_text), "characters")
print("\n----- Summarized Text -----")
print(summary)
print("Summarized text length:", len(summary), "characters")

## Step 5: Create the Translation Tool

Now let's build our second tool - a translation tool that can convert text into different languages. This will help make environmental information accessible to people around the world.

In [ ]:
# Create a translation tool
translate_prompt = PromptTemplate(
    input_variables=["text", "language"],
    template="""
    Translate the following text into {language}:

    {text}

    Translation:
    """
)

# Create a chain for translation
translate_chain = LLMChain(llm=llm, prompt=translate_prompt)

# Define a function that the agent can call
def translate_text(input_str):
    """Translate text to another language. Format: 'text to translate | target language'"""
    try:
        text, language = input_str.split("|")
        text = text.strip()
        language = language.strip()
        return translate_chain.run(text=text, language=language)
    except ValueError:
        return "Error: Please use the format 'text to translate | target language'"

print("✅ Translation tool created!")

## Step 6: Test the Translation Tool

Let's test our translation tool with the summary we created earlier.

In [ ]:
# Test our translation tool with the summary
# Format: "text to translate | target language"
translation_input = f"{summary} | Spanish"
spanish_translation = translate_text(translation_input)

print("----- Spanish Translation -----")
print(spanish_translation)

# Try another language
translation_input = f"{summary} | French"
french_translation = translate_text(translation_input)

print("\n----- French Translation -----")
print(french_translation)

## Step 7: Create the Agent with Both Tools

Now that we have our tools working individually, let's combine them into a single agent that can decide which tool to use based on user requests.

In [ ]:
# Define our tools in a format the agent can understand
tools = [
    Tool(
        name="Summarize",
        func=summarize_text,
        description="Useful for summarizing environmental text into bullet points. Input should be the text to summarize."
    ),
    Tool(
        name="Translate",
        func=translate_text,
        description="Useful for translating text to another language. Input should be in format: 'text to translate | target language'"
    )
]

# Create the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  # Shows the agent's thinking process
)

print("✅ Agent created with both summarization and translation tools!")

## Step 8: Test the Agent with Complex Tasks

Let's test our agent with tasks that require multiple steps. We'll ask it to summarize text and then translate the summary.

In [ ]:
print("==== Environmental Summarize & Translate Agent ====")
print("This agent can summarize environmental text and translate content to different languages.")
print("Let's test it with a multi-step task:\n")

# New environmental text about climate change
climate_text = """
Climate change is affecting ecosystems worldwide at an unprecedented rate. Global temperatures have risen by 1.1°C 
since pre-industrial times, with the last decade being the warmest on record. Ocean acidification has increased by 
30% since the Industrial Revolution due to CO2 absorption. Extreme weather events like hurricanes, floods, and 
wildfires have become more frequent and severe. These changes are causing mass migrations of species, with an 
estimated 1 million species at risk of extinction. However, recent international agreements have set targets to 
limit warming to 1.5°C, and over 70 countries have pledged carbon neutrality by 2050. Technological innovations in 
carbon capture, renewable energy, and sustainable agriculture offer promising solutions if implemented at scale.
"""

# Test the agent - summarize first, then translate to German
try:
    response = agent.run(f"First summarize this environmental text into bullet points, then translate the summary to German: {climate_text}")
    print("\nAgent completed the task successfully!")
except Exception as e:
    print(f"\nAn error occurred: {e}")
    print("Try running the example again or check your API key.")

## Step 9: Try Different Tasks

Now that we've seen how the agent works, try giving it different tasks. You can:
- Ask it to summarize different environmental texts
- Request translations to different languages
- Try more complex multi-step tasks

Here are some examples to try:

In [ ]:
# Example 1: Just summarize
print("=== Example 1: Just Summarize ===")
biodiversity_text = """
Biodiversity loss is accelerating at an alarming rate, with current extinction rates 100-1000 times higher than 
natural background rates. Habitat destruction, primarily from agriculture and urban expansion, remains the leading 
cause, affecting 85% of threatened species. Climate change is expected to become the leading driver of biodiversity 
loss by 2070 if current trends continue. Protected areas now cover 15% of land and 7% of marine environments, but 
many lack effective management. Conservation scientists advocate for protecting at least 30% of land and ocean by 
2030 to maintain ecological balance. Recent successes include the recovery of certain whale populations and the 
return of species previously thought extinct, demonstrating that targeted conservation efforts can be effective.
"""

try:
    response = agent.run(f"Summarize this text about biodiversity loss: {biodiversity_text}")
    print("\nTask completed!")
except Exception as e:
    print(f"\nAn error occurred: {e}")

In [ ]:
# Example 2: Just translate
print("=== Example 2: Just Translate ===")
green_energy_text = """
Renewable energy technologies are becoming increasingly cost-competitive with fossil fuels. Solar and wind power 
are now the cheapest forms of new electricity generation in most countries. This economic shift is accelerating 
the global energy transition.
"""

try:
    response = agent.run(f"Translate this text to Japanese: {green_energy_text}")
    print("\nTask completed!")
except Exception as e:
    print(f"\nAn error occurred: {e}")

## Your Turn: Create Your Own Tasks

Now it's your turn to experiment with the agent. Try creating your own environmental texts or using real-world content about sustainability topics. Ask the agent to:

1. Summarize complex environmental reports
2. Translate sustainability information to different languages
3. Perform multi-step tasks combining summarization and translation

**Challenge**: Try to create tasks that push the limits of the agent's capabilities!

In [ ]:
# Your code here
# Try running: agent.run("Your instructions here")

# Example you can modify:
"""
agent.run('''
    First summarize this text about sustainable agriculture, then translate the summary to Italian:
    
    Sustainable agriculture focuses on producing food while ensuring economic viability, 
    environmental stewardship, and social responsibility. Practices include crop rotation, 
    reduced tillage, precision farming, and integrated pest management. These methods help 
    preserve soil health, conserve water, reduce chemical inputs, and promote biodiversity. 
    Studies show that sustainable farms can achieve yields comparable to conventional farms 
    while using fewer resources and generating less pollution. Additionally, sustainable 
    agriculture often improves farmer livelihoods through premium pricing and reduced input costs.
''')
"""

## Congratulations!

You've successfully built and tested an AI agent with specialized tools for summarizing environmental content and translating it into multiple languages. This type of agent can be extremely useful for:

- Making complex environmental information more accessible
- Breaking down language barriers in global sustainability efforts
- Helping researchers quickly process large volumes of text
- Creating multilingual educational materials about climate change

### Key Takeaways:
- Custom tools give agents specialized capabilities
- Agents can reason about which tools to use based on the task
- Multi-step processing allows for complex workflows
- Language models can be tailored for specific domains like environmental science

### Next Steps:
- Try creating additional tools for your agent (e.g., fact-checking, visualization)
- Integrate real-world environmental data sources
- Customize the tools for specific sustainability topics
- Share your agent with others working on environmental challenges

Happy coding and thank you for contributing to environmental communication!